# DETR Knowledge Distillation on KITTI Dataset

This notebook demonstrates the complete pipeline for training a distilled DETR model on the KITTI dataset.

## Overview

1. **Setup**: Install dependencies and import libraries
2. **Configuration**: Load from YAML file
3. **Data Preparation**: Download and convert KITTI to COCO format
4. **Dataset Loading**: Create PyTorch datasets
5. **Model Setup**: Load teacher and student models
6. **Training**: Train with knowledge distillation
7. **Evaluation**: Evaluate with COCO metrics
8. **Visualization**: Visualize predictions

## 1. Setup and Installation

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

# Mount Google Drive if in Colab
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
!pip install -q torch torchvision transformers pycocotools pillow tqdm pyyaml matplotlib opencv-python

In [ ]:
# Setup working directory
import os
if IN_COLAB:
    if not os.path.exists('object-detection'):
        !git clone https://github.com/your-repo/object-detection.git
        %cd object-detection
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
print(f"Working directory: {os.getcwd()}")

## 2. Import Libraries

In [ ]:
import sys
import json
import random
import yaml
from pathlib import Path

import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm.auto import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torch.utils.data import DataLoader

from src.datasets import build_kitti_coco_dataset, collate_fn
from src.models import build_teacher_student_models
from src.distillation import DistillationLoss, DistillationTrainer
from src.utils import get_device, seed_all

print("✓ All imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {get_device()}")

## 3. Load Configuration from YAML

Load base configuration from `configs/distillation.yaml` with notebook-specific overrides for faster demo.

In [ ]:
# Load configuration from YAML file
config_path = Path('configs/distillation.yaml')
with open(config_path) as f:
    config = yaml.safe_load(f)

# Create flattened CONFIG for easier access
CONFIG = {
    'kitti_root': './kitti_data/training',
    'data_root': config['data']['root'],
    'output_dir': './output/distillation_notebook',
    'num_labels': config['data']['num_labels'],
    'train_split': 0.8,
    'max_samples': 200,
    'teacher_model': config['model']['teacher'],
    'student_model': config['model']['student'],
    'batch_size': config['data']['batch_size'],
    'num_workers': config['data']['num_workers'],
    'epochs': 3,
    'learning_rate': config['training']['learning_rate'],
    'weight_decay': config['training']['weight_decay'],
    'temperature': config['distillation']['temperature'],
    'alpha': config['distillation']['alpha'],
    'seed': 42,
    'device': None,
}

print("📋 Configuration loaded from:", config_path)
print("\n🔧 Notebook overrides (for faster demo):")
print(f"  • epochs: {config['training']['epochs']} → {CONFIG['epochs']}")
print(f"  • max_samples: full dataset → {CONFIG['max_samples']}")
print(f"  • output_dir: {config['output_dir']} → {CONFIG['output_dir']}")

seed_all(CONFIG['seed'])
device = get_device(CONFIG['device'])
print(f"\n🖥️  Using device: {device}")

Path(CONFIG['output_dir']).mkdir(parents=True, exist_ok=True)

print("\n📊 Active Configuration:")
print(f"  Teacher: {CONFIG['teacher_model']}")
print(f"  Student: {CONFIG['student_model']}")
print(f"  Epochs: {CONFIG['epochs']}")
print(f"  Batch size: {CONFIG['batch_size']}")
print(f"  Learning rate: {CONFIG['learning_rate']}")
print(f"  Temperature: {CONFIG['temperature']}")
print(f"  Alpha: {CONFIG['alpha']}")